Let's assume we calculated correlation matrix of 3 assets and the matrix is negative definite, which is a 

To use this correlation matrix to use Choletsky decomposition to simulated correlated random variables, we need to find the Positive Semi-Definite Matrix (PSD) that is closest to the given one.

In [3]:
import numpy as np

def isPSD(PSD): # is positive semi-definite?
    eigenvalues, _ = np.linalg.eigh(PSD)
    if np.all(eigenvalues > 0):
        return True
    else:
        return False
    
# Example of negative-definite matrix
M = np.array([[1, 0.5, 0.3],
              [0.5, 1, -0.7],
              [0.3, -0.7, 1]])

print('matrix determinant is negative:', np.linalg.det(M))
print('is matrix PSD?', isPSD(M))

matrix determinant is negative -0.03999999999999994
is matrix PSD? False


To find closest PSD matrix, we use Rebonato & Jäckel method

In [17]:
def find_nearest_psd_matrix(M):
    # Rebonato & Jäckel 1999
    # https://papers.ssrn.com/sol3/papers.cfm?abstract_id=1969689
    # accurate approximation to the closest matrix but not ideal (can write minimum-MSE-optimizer starting from this B * B.T solution)
    EPSILON = 1e-3
    n = M.shape[0]
    eigval, eigvec = np.linalg.eig(M) # C*S = LambdaDiagMatrix * S , where S is a matrix of eigenvectors
    val = np.matrix(np.maximum(eigval, EPSILON)) # correct eigenvalues to be positive
    vec = np.matrix(eigvec)
    T = 1 / (np.multiply(vec, vec) * val.T) # array to modify S matrix to have ones on the diagonal
    T = np.matrix(np.sqrt(np.diag(np.array(T).reshape((n))))) # T*S is the new eigenvector matrix that matches new eigenvalues
    B = T * vec * np.diag(np.array(np.sqrt(val)).reshape((n))) # B = T * S * diag(sqrt(non_neg_lambdas))
    return B * B.T  # C_new = B * B.T

psdm = find_nearest_psd_matrix(M)
print('original matrix \n', M)
print('------------------')
print('closest PSD matrix \n', np.around(psdm, 3))
print('matrix determinant is positive: {:.2}'.format(np.linalg.det(psdm)))
print('is matrix PSD?', isPSD(psdm))
print('difference between matrices: \n', np.around(psdm - M, 3))

original matrix 
 [[ 1.   0.5  0.3]
 [ 0.5  1.  -0.7]
 [ 0.3 -0.7  1. ]]
------------------
closest PSD matrix 
 [[ 1.     0.491  0.293]
 [ 0.491  1.    -0.688]
 [ 0.293 -0.688  1.   ]]
matrix determinant is positive: 0.0022
is matrix PSD? True
difference between matrices: 
 [[ 0.    -0.009 -0.007]
 [-0.009 -0.     0.012]
 [-0.007  0.012  0.   ]]
